# Assignment set 1

In [ ]:
from src.exercise1_1 import animate_wave, plotting_snapshots, calculate_wave
from src.exercise1_2 import analytical_vs_experimental, plot_heatmap, animate_diffusion
from src.exercise1_6 import *
from IPython.display import HTML
import matplotlib.pyplot as plt

plt.rcParams['animation.embed_limit'] = 50

#### Exercise 1.1

In [ ]:
L = 1
c = 1
N = 500
dt = 0.001
timesteps = 1000
initial_cond = 3
snapshot_arr = [0, 100, 200, 300, 400]
wave_data, x = calculate_wave(L, c, N, dt, timesteps, initial_cond)

In [ ]:
plotting_snapshots(wave_data, x, snapshot_arr)

In [ ]:
ani = animate_wave(wave_data, x, timesteps)
HTML(ani.to_jshtml())

#### Exercise 1.2

In [ ]:
N = 100
dt = 0.00001
dx = 1/N
D = 1
test_times = [0, 0.001, 0.01, 0.1, 1]
num_frames = 100

if 4*dt*D/(dx**2) <= 1:
    print('Stable')
else:
    raise ValueError('Not stable, reduce dt')

In [ ]:
analytical_vs_experimental(test_times, N, dt, dx, D)

In [ ]:
plot_heatmap(test_times[-3], N, dt, dx, D)

In [ ]:
ani = animate_diffusion(num_frames, N, dt, dx, D)
HTML(ani.to_jshtml())

In [ ]:
N = 100
eps = 1e-5
max_iter = int(2e4)
omegas = np.linspace(1.7, 2, 100)
Ns = np.linspace(10, 100, 10)

# per row: i_min, i_max, j_min, j_max
one_square = np.array([[18, 22, 48, 52]])

two_square = np.array([[18, 22, 31, 35],
                       [18, 22, 64, 68]])

three_square = np.array([[18, 22, 23, 27],
                         [18, 22, 48, 52],
                         [18, 22, 73, 77]])

In [ ]:
# Compare the linear behaviour of the concentration for multiple iterative schemes
compare_methods_to_analytical(N, 1.95, eps, max_iter)

In [ ]:
# The convergence versus the number of iterations for different numerical methods (and different values of omega for SOR)
convergence_measure(N, [1.75, 1.85, 1.95], eps, max_iter)

In [ ]:
# Finding the optimal omega for the SOR method and how it changes with N
optimal_omega(Ns, omegas, eps, max_iter)

In [ ]:
# The convergence versus the number of iterations for a different number of squares in the grid, N=100, omega=1.95
convergence_with_objects([one_square, two_square, three_square], N, 1.95, eps, max_iter)

In [ ]:
# The effect of objects on the optimal value for omega, N = 100
opt_omega = optimal_omega([100], omegas, eps, max_iter, plot=False)
print(f'The optimal omega without squares is {opt_omega[0]}')
optimal_omega_with_objects([one_square, two_square, three_square], N, omegas, eps, max_iter)

In [ ]:
# Plot the final grid with the three squares, N=100, omega=1.95
final_grid, _, _ = SOR_object(init_objects(three_square, N), N, 1.95, eps, max_iter)
plt.figure(figsize=(7, 5))
plt.imshow(final_grid, extent=[0, 1, 0, 1])
plt.xlabel('x', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.title(f'Diffusion in 2D with concentration sinks', fontsize=15)
plt.colorbar(label='Concentration')
plt.tight_layout()
plt.show()